In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import os  
import shutil
import random
from sklearn.metrics import mean_squared_error
import scipy.stats as st 
import tqdm
import pickle
import re

plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 600
plt.style.use('ggplot')
random.seed(100)

In [2]:
os.getcwd()

'/Users/jakegwinn/Documents/argonne_all/argonne'

In [4]:
df = pd.read_hdf("/Users/jakegwinn/Documents/argonne_all/merged.landmark.h5")

In [5]:
df.describe()

,auc,ic50,ec50,ec50se,r2fit,einf,hs,aac1,auc1,dss1,...,mordred.SRW10,mordred.TSRW10,mordred.MW,mordred.AMW,mordred.WPath,mordred.WPol,mordred.Zagreb1,mordred.Zagreb2,mordred.mZagreb1,mordred.mZagreb2
count,587709.000000,3.592340e+05,587709.000000,5.877090e+05,587709.000000,587709.000000,587709.000000,587709.000000,587709.000000,587709.000000,...,5.877090e+05,5.877090e+05,5.877090e+05,5.877090e+05,5.877090e+05,5.877090e+05,5.877090e+05,5.877090e+05,5.877090e+05,5.877090e+05
mean,0.841476,-4.057664e+16,5.285359,3.404194e+08,0.651540,0.378942,1.938250,0.153526,0.846474,0.118706,...,8.343823e-16,1.124374e-17,1.248055e-16,1.561550e-16,8.221230e-19,-1.120022e-16,1.203503e-16,7.183904e-17,-7.222955e-16,-2.400599e-16
std,0.154338,1.175797e+19,2.232449,5.062547e+10,0.491682,0.384477,1.546722,0.170102,0.170102,0.166824,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,0.000000,-6.173000e+21,0.000000,0.000000e+00,-183.700000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-9.781134e+00,-2.965784e+00,-1.512820e+00,-1.326300e+00,-1.049022e-01,-1.862548e+00,-2.025635e+00,-2.067898e+00,-1.109728e+00,-1.710678e+00
25%,0.777000,4.350000e+00,4.343000,5.070000e-02,0.293200,0.000000,0.530300,0.033100,0.785200,0.000000,...,-3.607745e-01,-5.447924e-01,-4.535860e-01,-4.448875e-01,-1.049020e-01,-5.298087e-01,-5.010291e-01,-5.023510e-01,-4.298036e-01,-4.476870e-01
50%,0.886900,5.067000e+00,5.330000,2.558000e-01,0.868100,0.239600,1.533000,0.093400,0.906600,0.046700,...,8.153928e-02,1.988934e-03,-1.166061e-01,-1.481723e-01,-1.049018e-01,-1.089437e-01,-8.299202e-02,-9.530886e-02,-1.527218e-01,-1.182112e-01
75%,0.957100,5.910000e+00,6.511000,2.291000e+00,0.970400,0.801800,4.000000,0.214800,0.966900,0.173400,...,4.517219e-01,3.762464e-01,2.106573e-01,1.709144e-01,-1.049016e-01,2.417772e-01,2.858642e-01,3.012963e-01,1.172029e-01,2.112646e-01
max,1.000000,1.200000e+01,12.000000,1.785000e+13,1.000000,0.999800,4.000000,1.000000,1.000000,1.000000,...,3.049585e+00,9.945969e+00,1.164647e+01,1.095231e+01,2.955059e+01,9.430663e+00,1.044171e+01,9.705014e+00,1.221269e+01,1.184490e+01


In [18]:
df[df["improve_chem_id"] == "Drug_988"]

,source,improve_chem_id,study,auc,ic50,ec50,ec50se,r2fit,einf,hs,...,mordred.SRW10,mordred.TSRW10,mordred.MW,mordred.AMW,mordred.WPath,mordred.WPol,mordred.Zagreb1,mordred.Zagreb2,mordred.mZagreb1,mordred.mZagreb2
93057,CTRPv2,Drug_988,605035,0.4095,7.823,7.869,0.0488,0.9836,0.0843,1.7340,...,-0.681016,-0.775955,-0.340953,-0.461364,-0.104902,-0.529809,-0.451848,-0.554536,-0.091375,-0.204502
487318,GDSCv2,Drug_988,14265,0.5452,6.645,6.645,7.4510,0.5833,0.0000,0.2924,...,-0.681016,-0.775955,-0.340953,-0.461364,-0.104902,-0.529809,-0.451848,-0.554536,-0.091375,-0.204502
37221,CTRPv2,Drug_988,312096,0.4174,7.990,8.031,0.0147,0.9967,0.1328,3.2890,...,-0.681016,-0.775955,-0.340953,-0.461364,-0.104902,-0.529809,-0.451848,-0.554536,-0.091375,-0.204502
243466,CTRPv2,Drug_988,354724,0.3356,7.987,7.987,0.0630,0.9800,0.0000,1.5070,...,-0.681016,-0.775955,-0.340953,-0.461364,-0.104902,-0.529809,-0.451848,-0.554536,-0.091375,-0.204502
297912,gCSI,Drug_988,237723,0.3936,7.639,7.639,0.0722,0.9875,0.0000,1.3440,...,-0.681016,-0.775955,-0.340953,-0.461364,-0.104902,-0.529809,-0.451848,-0.554536,-0.091375,-0.204502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188456,CTRPv2,Drug_988,294202,0.2826,8.380,8.385,0.0372,0.9697,0.0183,3.7850,...,-0.681016,-0.775955,-0.340953,-0.461364,-0.104902,-0.529809,-0.451848,-0.554536,-0.091375,-0.204502
570485,GDSCv2,Drug_988,12166,0.2625,8.540,8.545,0.0718,0.9913,0.0265,4.0000,...,-0.681016,-0.775955,-0.340953,-0.461364,-0.104902,-0.529809,-0.451848,-0.554536,-0.091375,-0.204502
553020,GDSCv2,Drug_988,11733,0.3335,8.001,8.001,0.9733,0.8030,0.0000,1.1890,...,-0.681016,-0.775955,-0.340953,-0.461364,-0.104902,-0.529809,-0.451848,-0.554536,-0.091375,-0.204502
498870,GDSCv2,Drug_988,17628,0.4030,7.582,7.582,579.2000,0.8592,0.0000,4.0000,...,-0.681016,-0.775955,-0.340953,-0.461364,-0.104902,-0.529809,-0.451848,-0.554536,-0.091375,-0.204502
